In [ ]:
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe
%use kandy

import ro.jf.funds.client.notebook.*
import ro.jf.funds.reporting.api.model.*

val username = "Johann-28.0"

val monthlyReportFrom = YearMonthTO(2019, 1)
val monthlyReportTo = YearMonthTO(2021, 4)
val monthlyReportForecastUntil = YearMonthTO(2021, 10)

val yearlyReportFrom = 2019
val yearlyReportTo = 2020
val yearlyReportForecastUntil = 2022

val fundName = "Work Income"
val reportViewName = "Work income report"

val REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/work-report-data-configuration.yaml"

val client = FundsClient()

In [ ]:
val user = client.ensureUserExists(username)
user

In [ ]:
val reportView = client.createReportView(user, reportViewName, fundName, client.fromYaml(File(REPORT_DATA_CONFIGURATION_YAML_FILE), "dataConfiguration"))
reportView

In [ ]:
val monthlyReportData = client.getMonthlyReportViewData(user, reportViewName, monthlyReportFrom, monthlyReportTo, monthlyReportForecastUntil)


In [ ]:
val yearlyReportData = client.getYearlyReportViewData(user, reportViewName, yearlyReportFrom, yearlyReportTo, yearlyReportForecastUntil)


In [ ]:
import kotlinx.datetime.DateTimeUnit
import kotlinx.datetime.TimeZone
import kotlinx.datetime.atStartOfDayIn
import kotlinx.datetime.minus
import org.jetbrains.kotlinx.kandy.ir.Plot
import java.math.BigDecimal

fun plotGroupedNetData(reportData: ReportDataTO, plotTitle: String, incomeGroupName: String, taxesGroup: String): Plot {
    val dataFrame = dataFrameOf(
        "bucket" to reportData.data.map<ReportDataItemTO, LocalDate> { it.timeBucket.from },
        "income" to reportData.data.map { bucket ->
            bucket.groupedNet!!.firstOrNull { it.group == incomeGroupName }?.net ?: BigDecimal.ZERO
        },
        "taxes" to reportData.data.map { bucket ->
            bucket.groupedNet!!.firstOrNull { it.group == taxesGroup }?.net ?: BigDecimal.ZERO
        },
        "net" to reportData.data.map { bucket ->
            bucket.groupedNet!!.mapNotNull { it.net }.fold(BigDecimal.ZERO) { acc, value -> acc + value }
        }
    )

    return dataFrame
        .plot {
            line {
                val values = dataFrame.get("net").values.map { it as BigDecimal }
                val forecastBorderMin = minOf(BigDecimal.ZERO, values.minOrNull() ?: BigDecimal.ZERO)
                val forecastBorderMax = values.maxOrNull() ?: BigDecimal.ZERO
                val forecastBorderX = when (reportData.interval.granularity) {
                    TimeGranularityTO.YEARLY -> reportData.interval.toDate.minus(183, DateTimeUnit.DAY)
                    TimeGranularityTO.MONTHLY -> reportData.interval.toDate.minus(15, DateTimeUnit.DAY)
                    TimeGranularityTO.DAILY -> reportData.interval.toDate.minus(1, DateTimeUnit.DAY)
                }

                y(listOf(forecastBorderMin, forecastBorderMax))
                x.constant(forecastBorderX.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds())
            }
            x("bucket") {
                val format = when (reportData.interval.granularity) {
                    TimeGranularityTO.YEARLY -> "%Y"
                    TimeGranularityTO.MONTHLY -> "%b %Y"
                    TimeGranularityTO.DAILY -> "%d %b %Y"
                }
                axis.breaks(reportData.data.map { it.timeBucket.from.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds() }, format)
            }
            line {
                y.constant(0)
            }
            line {
                y("net")
                color = Color.YELLOW
            }
            line {
                y("taxes")
                color = Color.RED
            }
            line {
                y("income")
                color = Color.GREEN
            }
            layout {
                title = plotTitle
                size = 2400 to 1200
            }
        }

}

In [ ]:
plotGroupedNetData(yearlyReportData, "Net income per year", "income", "taxes")

In [ ]:
plotGroupedNetData(monthlyReportData, "Income per month", "income", "taxes")